In [1]:
import numpy as np

epochs_path = "C:/Users/HYUN/capstone/LBM/processed_data_ica/all_epochs_ica_std.npy"
labels_path = "C:/Users/HYUN/capstone/LBM/processed_data_ica/all_labels_ica.npy"

epochs = np.load(epochs_path)
labels = np.load(labels_path, allow_pickle=True)  

print(f"epochs shape: {epochs.shape}")  # eg: (3621, 30, 512)
print(f"labels shape: {labels.shape}")  # eg: (3621,)

print(f"Number of samples: {epochs.shape[0]}")


epochs shape: (26508, 30, 1001)
labels shape: (26508,)
Number of samples: 26508


In [8]:
import torch
import numpy as np
from model_v2 import VQVAE 
import torch.nn.functional as F

MODEL_PATH = "C:/Users/HYUN/capstone/LBM/training/vq_vae_model_v2.pth"
DATA_PATH = "C:/Users/HYUN/capstone/LBM/processed_data_ica/all_epochs_ica_std.npy"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


embedding_dim = 256
input_channels = 30               
num_embeddings = 512
commitment_cost = 0.25
decay = 0.99

vqvae = VQVAE(input_channels, embedding_dim, num_embeddings, commitment_cost, decay).to(DEVICE)
vqvae.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
vqvae.eval()

data = np.load(DATA_PATH)
sample = torch.tensor(data[:8], dtype=torch.float32).to(DEVICE) 

with torch.no_grad():
    vq_loss, recon, perplexity, _ = vqvae(sample)
    recon_loss = F.mse_loss(recon, sample)
    total_loss = vq_loss + recon_loss

print(f"Reconstruction Loss (MSE): {recon_loss.item():.4f}")
print(f"VQ Loss (codebook + commitment): {vq_loss.item():.4f}")
print(f"Total Loss: {total_loss.item():.4f}")
print(f"Perplexity: {perplexity.item():.4f}")


C:\Users\HYUN\AppData\Local\Temp\ipykernel_39252\655955237.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vqvae.load_state_dict(torch.load(MODEL_PATH, map_location=DEV

Reconstruction Loss (MSE): 0.0929
VQ Loss (codebook + commitment): 0.2051
Total Loss: 0.2980
Perplexity: 355.1917
